# LSTM을 활용한 교통 변화 예측

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 

In [8]:
traffic_data_2015_01=pd.read_csv('data/2015_01분기.csv',encoding='cp949')
traffic_data_2015_02=pd.read_csv('data/2015_02분기.csv',encoding='cp949')
traffic_data_2015_03=pd.read_csv('data/2015_03분기.csv',encoding='cp949')
traffic_data_2015_04=pd.read_csv('data/2015_04분기.csv',encoding='cp949')
traffic_data_2016_01=pd.read_csv('data/2016_01분기.csv',encoding='cp949')
traffic_data_2016_02=pd.read_csv('data/2016_02분기.csv',encoding='cp949')
traffic_data_2016_03=pd.read_csv('data/2016_03분기.csv',encoding='cp949')
traffic_data_2016_04=pd.read_csv('data/2016_04분기.csv',encoding='cp949')
traffic_data_2017_01=pd.read_csv('data/2017_01분기.csv',encoding='cp949')
traffic_data_2017_02=pd.read_csv('data/2017_02분기.csv',encoding='cp949')
traffic_data_2017_03=pd.read_csv('data/2017_03분기.csv',encoding='cp949')
traffic_data_2017_04=pd.read_csv('data/2017_04분기.csv',encoding='cp949')
traffic_data_2018_01=pd.read_csv('data/2018_01분기.csv',encoding='cp949')
traffic_data_2018_02=pd.read_csv('data/2018_02분기.csv',encoding='cp949')
traffic_data_2018_03=pd.read_csv('data/2018_03분기.csv',encoding='cp949')
traffic_data_2018_04=pd.read_csv('data/2018_04분기.csv',encoding='cp949')
traffic_data_2019_01=pd.read_csv('data/2019_01분기.csv',encoding='cp949')
traffic_data_2019_02=pd.read_csv('data/2019_02분기.csv',encoding='cp949')
traffic_data_2019_03=pd.read_csv('data/2019_03분기.csv',encoding='cp949')
traffic_data_2019_04=pd.read_csv('data/2019_04분기.csv',encoding='cp949')
traffic_data_2020_01=pd.read_csv('data/2020_01분기.csv',encoding='cp949')
traffic_data_2020_02=pd.read_csv('data/2020_02분기.csv',encoding='cp949')
traffic_data_2020_03=pd.read_csv('data/2020_03분기.csv',encoding='cp949')
traffic_data_2020_04=pd.read_csv('data/2020_04분기.csv',encoding='cp949')
traffic_data_2021_01=pd.read_csv('data/2021_01분기.csv',encoding='cp949')
traffic_data_2021_02=pd.read_csv('data/2021_02분기.csv',encoding='cp949')

In [9]:
traffic_data_2015=pd.concat([traffic_data_2015_01,traffic_data_2015_02,traffic_data_2015_03,traffic_data_2015_04])
traffic_data_2016=pd.concat([traffic_data_2016_01,traffic_data_2016_02,traffic_data_2016_03,traffic_data_2016_04])
traffic_data_2017=pd.concat([traffic_data_2017_01,traffic_data_2017_02,traffic_data_2017_03,traffic_data_2017_04])
traffic_data_2018=pd.concat([traffic_data_2018_01,traffic_data_2018_02,traffic_data_2018_03,traffic_data_2018_04])
traffic_data_2019=pd.concat([traffic_data_2019_01,traffic_data_2019_02,traffic_data_2019_03,traffic_data_2019_04])
traffic_data_2020=pd.concat([traffic_data_2020_01,traffic_data_2020_02,traffic_data_2020_03,traffic_data_2020_04])
traffic_data_2021=pd.concat([traffic_data_2021_01,traffic_data_2021_02])
traffic_data_past=pd.concat([traffic_data_2015,traffic_data_2016,traffic_data_2017,traffic_data_2018,traffic_data_2019])

In [10]:
traffic_data_past.head()

,집계일자,영업소코드,영업소명,입출구구분코드,입출구명,TCS하이패스구분코드,TCS하이패스명,고속도로운영기관구분코드,고속도로운영기관명,영업형태구분코드,영업형태명,1종교통량,2종교통량,3종교통량,4종교통량,5종교통량,6종교통량,총교통량
0,20150101,246,가락,0,입구,1,TCS,0,한국도로공사,0,폐쇄식,329,14,31,45,467,1,887
1,20150101,246,가락,0,입구,2,hi-pass,0,한국도로공사,0,폐쇄식,696,16,4,2,11,37,766
2,20150101,29,가락(개),0,입구,1,TCS,0,한국도로공사,1,개방식,2931,50,91,114,454,316,3956
3,20150101,29,가락(개),0,입구,2,hi-pass,0,한국도로공사,1,개방식,5219,20,24,0,1,283,5547
4,20150101,29,가락(개),1,출구,1,TCS,0,한국도로공사,1,개방식,3374,42,107,94,603,415,4635


In [12]:
traffic_data_past['집계일자']=pd.to_datetime(traffic_data_past['집계일자'],format='%Y%m%d')
traffic_data_past['연도']=traffic_data_past['집계일자'].dt.year

In [13]:
#plt.figure(figsize=(16, 9))
#sns.lineplot(y=traffic_data_past['1종교통량'], x=traffic_data_past['집계일자'])
#plt.xlabel('time')
#plt.ylabel('type 1 traffic')

In [14]:
traffic_data_past=traffic_data_past.replace('서서울영업소','서서울')
traffic_data_past=traffic_data_past.replace('서울영업소','서울')
traffic_data_past=traffic_data_past.replace('서대구영업소','서대구')
traffic_data_past=traffic_data_past.replace('북대구영업소','북대구')
traffic_data_past=traffic_data_past.replace('남대구영업소','남대구')

In [15]:
seoul_traffic_data_past=traffic_data_past[(traffic_data_past['영업소명']=='서서울')|(traffic_data_past['영업소명']=='서울')|(traffic_data_past['영업소명']=='남서울')
]

In [16]:
from sklearn.preprocessing import MinMaxScaler #정규화 과정

seoul_traffic_data_past.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['1종교통량','2종교통량','3종교통량','총교통량']
df_scaled = scaler.fit_transform(seoul_traffic_data_past[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

,1종교통량,2종교통량,3종교통량,총교통량
0,0.019727,0.000415,0.000352,0.004809
1,0.050587,0.000830,0.000216,0.012472
2,0.015767,0.000338,0.000343,0.004289
3,0.041785,0.000740,0.000212,0.010290
4,0.021762,0.000423,0.000520,0.005272
...,...,...,...,...
14547,0.072841,0.002691,0.001035,0.019906
14548,0.004851,0.000286,0.000382,0.001129
14549,0.068989,0.002575,0.005453,0.020292
14550,0.003695,0.000379,0.000341,0.001179


In [17]:
TEST_SIZE = 200
WINDOW_SIZE = 20

train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [18]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [19]:
from sklearn.model_selection import train_test_split

feature_cols = ['1종교통량','2종교통량','3종교통량']
label_cols = ['총교통량']

train_feature = train[feature_cols]
train_label = train[label_cols]

train_feature, train_label = make_dataset(train_feature, train_label, 20)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
x_train.shape, x_valid.shape

((11465, 20, 3), (2867, 20, 3))

In [20]:
test_feature = test[feature_cols]
test_label = test[label_cols]

test_feature.shape, test_label.shape

((200, 3), (200, 1))

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )

model.add(Dense(1))

In [22]:
import os

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train,
                                    epochs=100,
                                    batch_size=16,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])

Epoch 1/100
716/717 [============================>.] - ETA: 0s - loss: 1.6366e-04
Epoch 00001: val_loss improved from inf to 0.00001, saving model to model\tmp_checkpoint.h5
717/717 [==============================] - 6s 9ms/step - loss: 1.6353e-04 - val_loss: 8.3054e-06
Epoch 2/100
713/717 [============================>.] - ETA: 0s - loss: 1.4200e-04
Epoch 00002: val_loss did not improve from 0.00001
717/717 [==============================] - 6s 8ms/step - loss: 1.4151e-04 - val_loss: 3.0848e-05
Epoch 3/100
717/717 [==============================] - ETA: 0s - loss: 1.3490e-04
Epoch 00003: val_loss improved from 0.00001 to 0.00001, saving model to model\tmp_checkpoint.h5
717/717 [==============================] - 7s 9ms/step - loss: 1.3490e-04 - val_loss: 6.7969e-06
Epoch 4/100
716/717 [============================>.] - ETA: 0s - loss: 1.3523e-04
Epoch 00004: val_loss improved from 0.00001 to 0.00001, saving model to model\tmp_checkpoint.h5
717/717 [==============================] - 6s 

Epoch 33/100
711/717 [============================>.] - ETA: 0s - loss: 1.2892e-04
Epoch 00033: val_loss improved from 0.00000 to 0.00000, saving model to model\tmp_checkpoint.h5
717/717 [==============================] - 6s 8ms/step - loss: 1.2795e-04 - val_loss: 2.9200e-06
Epoch 34/100
711/717 [============================>.] - ETA: 0s - loss: 1.2922e-04
Epoch 00034: val_loss improved from 0.00000 to 0.00000, saving model to model\tmp_checkpoint.h5
717/717 [==============================] - 6s 8ms/step - loss: 1.2823e-04 - val_loss: 2.8957e-06
Epoch 35/100
715/717 [============================>.] - ETA: 0s - loss: 1.2861e-04
Epoch 00035: val_loss did not improve from 0.00000
717/717 [==============================] - 6s 9ms/step - loss: 1.2834e-04 - val_loss: 2.9218e-06
Epoch 36/100
715/717 [============================>.] - ETA: 0s - loss: 1.2854e-04
Epoch 00036: val_loss did not improve from 0.00000
717/717 [==============================] - 6s 9ms/step - loss: 1.2827e-04 - val_los